# Lab Assignment - Part 1
## Wide and Deep Recomendation System
### Shruti Badrinarayanan, Poojitha Venkatram

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pip install scikit-learn

In [ ]:
# pip install pandas

## Load Data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the datasets
books = pd.read_csv('/content/drive/MyDrive/Lab/Part 1/BX-Books.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')
users = pd.read_csv('/content/drive/MyDrive/Lab/Part 1/BX-Users.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')
ratings = pd.read_csv('/content/drive/MyDrive/Lab/Part 1/BX-Book-Ratings.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')

# Display the first few rows of each dataset to understand its structure
books.head(), users.head(), ratings.head()

<ipython-input-68-ba258d49bd75>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('/content/drive/MyDrive/Lab/Part 1/BX-Books.csv', error_bad_lines=False, delimiter=';', encoding = 'ISO-8859-1')
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 

(         ISBN                                         Book-Title  \
 0  0195153448                                Classical Mythology   
 1  0002005018                                       Clara Callan   
 2  0060973129                               Decision in Normandy   
 3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
 4  0393045218                             The Mummies of Urumchi   
 
             Book-Author Year-Of-Publication                   Publisher  \
 0    Mark P. O. Morford                2002     Oxford University Press   
 1  Richard Bruce Wright                2001       HarperFlamingo Canada   
 2          Carlo D'Este                1991             HarperPerennial   
 3      Gina Bari Kolata                1999        Farrar Straus Giroux   
 4       E. J. W. Barber                1999  W. W. Norton &amp; Company   
 
                                          Image-URL-S  \
 0  http://images.amazon.com/images/P/0195153448.0...   
 1  http://i

## Preprocess Data

In [ ]:
ratings_and_books = ratings.merge(books, on='ISBN', how='inner', suffixes=('_1', '_2'))
all = ratings_and_books.merge(users, on='User-ID', how='inner', suffixes=('_1', '_2'))
all_copy = all.copy()
all.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'User-ID', 'ISBN'], axis=1, inplace=True)

# Assuming books, users, and ratings have been filtered to only include relevant rows
# Fill NaN values with a placeholder
all['Book-Title'] = all['Book-Title'].fillna('Unknown')
all['Book-Author'] = all['Book-Author'].fillna('Unknown')
all['Age'] = all['Age'].fillna(int(all['Age'].mean())).values

all["Age"] = pd.to_numeric(all["Age"])

all.drop(all.loc[all['Year-Of-Publication']=='DK Publishing Inc'].index, inplace=True)
all.drop(all.loc[all['Year-Of-Publication']=='Gallimard'].index, inplace=True)

all["Year-Of-Publication"] = pd.to_numeric(all["Year-Of-Publication"])

In [ ]:
all['Age'] = all['Age'].astype('float32')

all.dtypes

Book-Rating              int64
Book-Title              object
Book-Author             object
Year-Of-Publication      int64
Publisher               object
Location                object
Age                    float32
dtype: object

In [ ]:
# all_sample = all.sample(n=100000)
all_sample = all

In [ ]:
all_sample

## Define BookDataset Class which helps initialise the Data

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

class BookDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe

        # Preprocess string features with categorical embeddings
        self.embedders = {}
        for col in ['Book-Title', 'Book-Author', 'Publisher', 'Location']:
            encoder = LabelEncoder()
            self.df[col] = encoder.fit_transform(self.df[col])
            self.embedders[col] = (encoder.classes_.size, min(10, len(encoder.classes_)//2)) # Adjust embedding size as needed

        # Target variable
        self.target = self.df['Book-Rating'].to_numpy()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        features = {
            col: torch.tensor(self.df[col].iloc[index]).long()
            for col in ['Book-Title', 'Book-Author', 'Publisher', 'Location', 'Year-Of-Publication', 'Age']
        }

        return features, torch.nn.functional.one_hot(torch.tensor(self.target[index]), 11)

In [ ]:
train_df = all_sample.sample(frac=0.8, random_state=42)
test_df = all_sample.drop(train_df.index)

train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

train_dataset = BookDataset(train_df_copy)
test_dataset = BookDataset(test_df_copy)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

## Define the Model

In [ ]:
import torch.nn as nn

class BookRatingModel(nn.Module):
    def __init__(self, embedding_dims):
        super().__init__()

        self.embedding_dims = embedding_dims

        # Embedding layers for categorical features
        self.embeddings = nn.ModuleDict({
            name: nn.Embedding(num_embeddings, dim)
            for name, (num_embeddings, dim) in embedding_dims.items()
        })

        # Dense layers for continuous features and embeddings
        self.deep_layers = nn.Sequential(
            nn.Linear(42, 128),  # Adjust sizes as needed
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 11)
        )

        self.wide_layer = nn.Linear(42, 11)

    def forward(self, x):
        embeddings = [self.embeddings[name](x[name]) for name in ['Book-Title', 'Book-Author', 'Publisher', 'Location']]
        embeddings = torch.cat(embeddings, dim=1)
        x = torch.cat([embeddings, x['Year-Of-Publication'].unsqueeze(1), x['Age'].unsqueeze(1)], dim=1)
        return self.deep_layers(x) + self.wide_layer(x)

# Initialize the model
embedding_dims = train_dataset.embedders  # Get embedding dimensions from the dataset
model = BookRatingModel(embedding_dims)

In [ ]:
def eval():
  # Evaluate the model
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for data in test_loader:
          features, targets = data
          outputs = model(features)
          _, predicted = torch.max(outputs.data, 1)

          for i, target in enumerate(targets):
            one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)
            correct += torch.all(torch.eq(targets[i], one_hot[i])).item()
          total += targets.size(0)
  print(f'Accuracy on test set: {round(correct / total * 100, 3)}%')

## Model Training

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.003)
loss_fn = nn.CrossEntropyLoss()  # Cross Entropy loss

for epoch in range(5):
    i = 0
    cum_loss = 0
    total_batches = len(train_loader)
    for features, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = loss_fn(outputs.squeeze(), targets.float())
        loss.backward()
        optimizer.step()
        i += 1
        cum_loss += loss
        # if i > 0 and i % 500 == 0:
        #   print('Loss=', loss.item(), 'in', i, '/', total_batches, 'batches', 'in epoch', epoch)
    avg_loss = cum_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss}")
    eval()

Epoch 1, Average Loss: 1.9596067667007446


/tmp/ipykernel_184/1175092157.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)


Accuracy on test set: 49.207%
Epoch 2, Average Loss: 1.667163372039795
Accuracy on test set: 57.359%
Epoch 3, Average Loss: 1.6333808898925781
Accuracy on test set: 42.978%
Epoch 4, Average Loss: 1.6002947092056274
Accuracy on test set: 20.536%
Epoch 5, Average Loss: 1.5620392560958862
Accuracy on test set: 54.305%


In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        features, targets = data
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)

        for i, target in enumerate(targets):
          one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)
          correct += torch.all(torch.eq(targets[i], one_hot[i])).item()
        total += targets.size(0)
print(f'Accuracy on train set: {round(correct / total * 100, 3)}%')

/tmp/ipykernel_184/22237457.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)


Accuracy on train set: 64.772%


In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        features, targets = data
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)

        for i, target in enumerate(targets):
          one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)
          correct += torch.all(torch.eq(targets[i], one_hot[i])).item()
        total += targets.size(0)
print(f'Accuracy on test set: {round(correct / total * 100, 3)}%')

/tmp/ipykernel_184/1239849378.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)


Accuracy on test set: 54.305%


## Save Model

In [ ]:
# Save the entire model (architecture + weights)
torch.save(model, 'complete_model.pth')

In [ ]:
model = torch.load('/content/drive/MyDrive/Lab/Part 1/complete_model.pth')

In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    count = 0
    for data in test_loader:
        features, targets = data
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        count += 32
        print("count:", count)

        for i, target in enumerate(targets):
          one_hot = torch.nn.functional.one_hot(torch.tensor(predicted), 11)
          correct += torch.all(torch.eq(targets[i], one_hot[i])).item()

          test_df.loc[count, 'Predicted Rating'] = int(predicted[i])
        total += targets.size(0)
print(f'Accuracy on test set: {round(correct / total * 100, 3)}%')

## Make Book Recomendations based on some Input Data

In [ ]:
books = test_df.loc[(test_df['Location'] == "colorado springs, colorado, usa") & (test_df['Age'] == 52.0)].nlargest(5, 'Predicted Rating')
print("Here are the top 5 five recommended books for user 198929 who is living in colorado springs, colorado, usa and their age is 52:\n")
count = 1
for i, book in books.iterrows():
  print(count, ":", book['Book-Title'], "by", book['Book-Author'])
  count += 1

Here are the top 5 five recommended books for user 198929 who is living in colorado springs, colorado, usa and their age is 52:

1 : Dance of the Flame by Elaine Barbieri
2 : House by Tracy Kidder
3 : The Saving Graces: A Novel by Patricia Gaffney
4 : How Do Dinosaurs Learn to Read by Jane Yolen
5 : Remote Control by Andy McNab


In [ ]:
books = test_df.loc[(test_df['Location'] == "ottawa, ontario, canada") & (test_df['Age'] == 32.0)].nlargest(5, 'Predicted Rating')
print("Here are the top 5 five books recommended for user 44839 who is living in ottawa, ontario, canada and their age is 32:\n")
count = 1
for i, book in books.iterrows():
  print(count, ":", book['Book-Title'], "by", book['Book-Author'])
  count += 1

Here are the top 5 five books recommended for user 44839 who is living in ottawa, ontario, canada and their age is 32:

1 : Burning Bright: A Play in Story Form (Penguin Twentieth Century Classics) by John Steinbeck
2 : The King of Torts by John Grisham
3 : One Flew Over the Cuckoo's Nest by Ken Kesey
4 : The Stand: Complete and Uncut by Stephen King
5 : Childhood's End by Arthur C. Clarke


In [ ]:
books = test_df.loc[(test_df['Location'] == "fairbanks, alaska, usa") & (test_df['Age'] == 30.0)].nlargest(5, 'Predicted Rating')
print("Here are the top 5 five books recommended for user 98741 who is living in fairbanks, alaska, usa and their age is 30:\n")
count = 1
for i, book in books.iterrows():
  print(count, ":", book['Book-Title'], "by", book['Book-Author'])
  count += 1

Here are the top 5 five books recommended for user 98741 who is living in fairbanks, alaska, usa and their age is 30:

1 : The Charmer by Patrick Hamilton
2 : America: The Vegetarian Table (Vegetarian Table) by Deborah Madison
3 : Native American Songs and Poems: An Anthology (Dover Thrift Editions) by Brian Swann
4 : The Diva Principle (Ay Insp - Hammond) by Michelle McKinney Hammond
5 : Why Do Clocks Run Clockwise? by David Feldman
